~~This runs our model with byt5-ocr specific corrections: https://huggingface.co/yelpfeast/byt5-base-english-ocr-correction ~~

Using the historical dataset for training.

Make sure you are in the PyTorchOCR environment on HAL!

In [1]:
main_dir = '/home/jnaiman/'
aligned_dataset_dir = '/home/jnaiman/wwt_image_extraction/alignments/'

historical_dataset_dir = '/home/jnaiman/wwt_image_extraction/historical_docs/groundtruth/'
types = ['plain', 'author location']
only_words = False

# which model do we want to start from pre-trained?
model_pretrained = 'google/byt5-small' # orig

# load model weights from prior checkpoint
#check_point_load = '/home/jnaiman/models/byt5_inline_cite_ref_large/checkpoint-87000'
#output_dir = main_dir + 'models/byt5_ocr_full_hal_historical_withPreTrain/' # math/cite/refs -- just left in as raw

# output_dir = main_dir + 'models/byt5_ocr_full_hal_historical_ocr/' # math/cite/refs -- just left in as raw
# model_pretrained = 'yelpfeast/byt5-base-english-ocr-correction' # for OCR correction specifically
# check_point_load = None # to start running

output_dir = main_dir + 'models/byt5_ocr_full_hal_historical/' # math/cite/refs -- just left in as raw
check_point_load = None # to start running


remake_splits = False
train_frac = 0.8
val_frac = 0.1
test_frac = 0.1

In [2]:
import pandas as pd
from glob import glob
import pickle
import time

In [3]:
hfiles = glob(historical_dataset_dir + '*.pickle')
len(hfiles)

216

In [4]:
filenames = []; pages = []; sents = []; 
source = []; target = []
types_here = []
#target_cleaned = []; source_cleaned = []
for hfile in hfiles:
    with open(hfile,'rb') as f:
        dir_test = pickle.load(f)

    filenames.append(dir_test['filename'])
    pages.append(dir_test['page'])
    sents.append(dir_test['sent'])
    source.append(dir_test['source'])
    target.append(dir_test['target'])
    types_here.append(dir_test['type'])
#     # cleaned
#     source_cleaned.append(dir_test['source'].replace('^',''))
#     target_cleaned.append(dir_test['target'].replace()
    

df_historical_test_all = pd.DataFrame({'input_text_unclean':source, 
                                       'target_text_unclean':target,
                                   'filename':filenames, 'page':pages,
                                   'sent num':sents, 'type':types_here})

df_historical_test = df_historical_test_all.loc[df_historical_test_all['type'].isin(types)]

In [5]:
if train_frac + val_frac + test_frac != 1.0:
    print('the sets fracs dont add to 1!')

Make splits of the data:

In [6]:
if remake_splits:
    train_df = df_historical_test.sample(frac=train_frac, replace=False)
    dftmp = df_historical_test[~df_historical_test.isin(train_df)].dropna()
    # split
    eval_df = dftmp.sample(frac=val_frac/(test_frac+val_frac), replace=False)
    test_df = dftmp[~dftmp.isin(eval_df)].dropna()

    # save all of these
    train_df.to_csv(aligned_dataset_dir + 'train_historical'+str(time.time())+'.csv',index=False)
    eval_df.to_csv(aligned_dataset_dir + 'val_historical'+str(time.time())+'.csv',index=False)
    test_df.to_csv(aligned_dataset_dir + 'test_historical'+str(time.time())+'.csv',index=False)
else: # use already there
    fs = glob(aligned_dataset_dir + 'train_historical*csv')
    fs.sort()
    fs = fs[-1] # most recent
    train_df = pd.read_csv(fs)
    fs = glob(aligned_dataset_dir + 'val_historical*csv')
    fs.sort()
    fs = fs[-1] # most recent
    eval_df = pd.read_csv(fs)
    fs = glob(aligned_dataset_dir + 'test_historical*csv')
    fs.sort()
    fs = fs[-1] # most recent
    test_df = pd.read_csv(fs)

In [7]:
#fs

In [8]:
eval_df.head(n=1)

,input_text_unclean,target_text_unclean,filename,page,sent num,type
0,The photograph in Figure 1 (Plate 1) was obtai...,The photograph in Figure 1 (Plate 1) was obtai...,1981ApJ___249__390M,1,0,plain


In [9]:
# do some re-naming
train_df = train_df.rename(columns={"input_text_unclean":"sentences source", 
                        "target_text_unclean":"sentences target"})
eval_df = eval_df.rename(columns={"input_text_unclean":"sentences source", 
                        "target_text_unclean":"sentences target"})
test_df = test_df.rename(columns={"input_text_unclean":"sentences source", 
                        "target_text_unclean":"sentences target"})

In [10]:
from transformers import HfArgumentParser, TensorFlowBenchmark, TensorFlowBenchmarkArguments
#import pandas as pd
from transformers import T5ForConditionalGeneration, AutoTokenizer
from transformers import TrainingArguments
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from transformers import EarlyStoppingCallback

In [11]:
#import fastwer
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

In [12]:
from sys import path
path.append(main_dir + 'libraries/')
from utils_ocr_mini import get_fill_in_types

In [13]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'
cuda.empty_cache()
print(device)

cuda


In [14]:
def add_formatted_columns(datain):
    source = []
    target = []
    source_aligned = []
    target_aligned = []
    for i in range(len(datain)):
        d = datain.iloc[i]
        s = np.array(list(d['aligned sentences source'])) # aligned source, with ^ symbols
        t = np.array(list(d['aligned sentences target'])) # aligned target, with @ symbols
        a = np.array(list(get_fill_in_types(d['aligned sentences target types'])))
        if len(s) == len(t):
            ss = "".join(s[np.where( (a == ' ') | (a == 'W') | (a == 'w'))[0]].tolist())
            tt = "".join(t[np.where( (a == ' ') | (a == 'W') | (a == 'w'))[0]].tolist())
        else:
            print('have issue, testing')
            if t[0] == ' ' and s[0] != ' ':
                t = np.array(list(d['aligned sentences target']))[1:] # aligned target, with @ symbols
                a = np.array(list(get_fill_in_types(d['aligned sentences target types'])))[1:]
                if len(s) == len(t):
                    ss = "".join(s[np.where( (a == ' ') | (a == 'W') | (a == 'w'))[0]].tolist())
                    tt = "".join(t[np.where( (a == ' ') | (a == 'W') | (a == 'w'))[0]].tolist())
                else:
                    print('not aligned, best guess')
                    import sys; sys.exit()

        source_aligned.append(ss.replace('^','@')) # align with original 
        target_aligned.append(tt)
        source.append(ss.replace('^',''))
        target.append(tt.replace('@',''))

    datain['words source aligned'] = source_aligned
    datain['words target aligned'] = target_aligned
    datain['words source'] = source
    datain['words target'] = target
    return datain

In [15]:
train_df.head()

,sentences source,sentences target,filename,page,sent num,type
0,The bulk properties of the outer and reverse s...,The bulk properties of the outer and reverse s...,1996ApJ___469__171G,7,38,plain
1,This temperature range is a factor of ~20 grea...,This temperature range is a factor of $\sim 20...,1996ApJ___470__513M,5,13,plain
2,y. The straight lines drawn represent: (a) the...,$_{V_\circ}$. The straight lines drawn represe...,1969AJ_____74__882W,7,20,plain
3,All catalogued nebulae showing on any of the p...,All catalogued nebulae showing on any of the p...,1917ApJ____46___24P,13,2,plain
4,"ture only, we must assume that, as we ascend t...","ture only, we must assume that, as we ascend t...",1913ApJ____38__407M,86,0,plain


In [16]:
if only_words:
    train_df = add_formatted_columns(train_df)
    eval_df = add_formatted_columns(eval_df)
    test_df = add_formatted_columns(test_df)
    # rename sentences we want
    train_df = train_df.rename(columns={"words source": "input_text", 
                        "words target": "target_text"})
    eval_df = eval_df.rename(columns={"words source": "input_text", 
                        "words target": "target_text"})
    test_df = test_df.rename(columns={"words source": "input_text", 
                        "words target": "target_text"})
else:
    # rename sentences we want
    train_df = train_df.rename(columns={"sentences source": "input_text", 
                        "sentences target": "target_text"})
    eval_df = eval_df.rename(columns={"sentences source": "input_text", 
                        "sentences target": "target_text"})
    test_df = test_df.rename(columns={"sentences source": "input_text", 
                        "sentences target": "target_text"})

In [17]:
args_dict = {
    #"model_name_or_path": 'google/byt5-small',
    #"max_len": 4096,
    #"max_length": 4096,
    "output_dir": output_dir,
    "logging_dir":output_dir + '/logs/',
    "overwrite_output_dir": True,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "learning_rate": 5e-4,
    "warmup_steps": 250,
    "logging_steps": 100,
    "evaluation_strategy": "steps",
    "eval_steps": 1000,
    "num_train_epochs": 4,
    "do_train": True,
    "do_eval": True,
    "fp16": False,
    #"use_cache": False,
    "max_steps": 100000,
    'save_steps':1000,
    'save_strategy':'steps',
    'load_best_model_at_end': True#,
    # 'metric_for_best_model':'eval_loss',
    # 'greater_is_better':False
}

In [18]:
#!pip install --upgrade accelerate

In [19]:
parser = HfArgumentParser(
        (TrainingArguments))
training_args = parser.parse_dict(args_dict)
# set_seed(training_args.seed)
args = training_args[0]

In [20]:
# Load pretrained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_pretrained,
    cache_dir=output_dir, 
    max_length=4096
)

Downloading:   0%|          | 0.00/2.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/698 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

In [21]:
model = T5ForConditionalGeneration.from_pretrained(
    model_pretrained,
    cache_dir=output_dir,
)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [22]:
# overwriting the default max_length of 20 
tokenizer.model_max_length=4096
model.config.max_length=4096

In [23]:
class GPReviewDataset(Dataset):
    def __init__(self, Text, Label):
        self.Text = Text
        self.Label = Label
        # self.tokenizer = tokenizer
        # self.max_len = max_len
    def __len__(self):
        return len(self.Text)
    def __getitem__(self, item):
        Text = str(self.Text[item])
        Label = self.Label[item]
        inputs = tokenizer(Text, padding="max_length", truncation=True, max_length=512)
        outputs = tokenizer(Label, padding="max_length", truncation=True, max_length=512)
        return {
          "input_ids":inputs.input_ids,
          "attention_mask" : inputs.attention_mask,
          "labels" : outputs.input_ids,
          "decoder_attention_mask" : outputs.attention_mask,
          # "labels" : lbz
        }

In [24]:
ds_train = GPReviewDataset(
  Text=train_df.input_text.to_numpy(),
  Label=train_df.target_text.to_numpy()
  # tokenizer=tokenizer,
  # max_len=max_len
)

In [25]:
ds_test = GPReviewDataset(
  Text=eval_df.input_text.to_numpy(),
  Label=eval_df.target_text.to_numpy()
  # tokenizer=tokenizer,
  # max_len=max_len
)

In [26]:
train_dataset = ds_train
valid_dataset = ds_test

In [27]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
    # compute_metrics=compute_metrics

)

/home/jnaiman/.conda/envs/PyTorchOCR/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
max_steps is given, it will override any value given in num_train_epochs


In [28]:
#!ls gdrive/MyDrive/TPDL\ 2023\ Colab\ Notebooks/byt5_models/byt5_inline_cite_ref_ocr

In [29]:
#output_dir

In [30]:
#f''+output_dir + 'checkpoint-15000'
#trainer.args?

In [31]:
trainer.args.save_total_limit = 20
trainer.args.logging_steps = 10 # down from 100
trainer.args.save_steps=10 # down from 10000
trainer.args.eval_steps = trainer.args.logging_steps # down from 1000
trainer.args.max_steps = 100000 # down from 10000
if check_point_load is None:
    trainer.train() # put in checkpoint if need be here to load 
else:
    trainer.train(check_point_load)

/home/jnaiman/.conda/envs/PyTorchOCR/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 86
  Num Epochs = 20000
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 100000


Step,Training Loss,Validation Loss
10,64.377300,77.402344
20,64.263300,71.808060
30,60.828400,63.747990
40,54.879700,53.098999
50,45.856500,38.290672
60,32.150000,23.657043
70,14.412200,4.541070
80,4.883400,4.141339
90,4.235800,3.494669
100,3.497000,2.358604


***** Running Evaluation *****
  Num examples = 11
  Batch size = 4
Saving model checkpoint to /home/jnaiman/models/byt5_ocr_full_hal_historical/checkpoint-10
Configuration saved in /home/jnaiman/models/byt5_ocr_full_hal_historical/checkpoint-10/config.json
Model weights saved in /home/jnaiman/models/byt5_ocr_full_hal_historical/checkpoint-10/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11
  Batch size = 4
Saving model checkpoint to /home/jnaiman/models/byt5_ocr_full_hal_historical/checkpoint-20
Configuration saved in /home/jnaiman/models/byt5_ocr_full_hal_historical/checkpoint-20/config.json
Model weights saved in /home/jnaiman/models/byt5_ocr_full_hal_historical/checkpoint-20/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11
  Batch size = 4
Saving model checkpoint to /home/jnaiman/models/byt5_ocr_full_hal_historical/checkpoint-30
Configuration saved in /home/jnaiman/models/byt5_ocr_full_hal_historical/checkpoint-30/config.json
Model weights sa

KeyboardInterrupt: 